In [10]:
import yfinance as yf
from datetime import datetime, timedelta


def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}


def get_stock_performance(ticker, days):
    """Method to get stock price change in percentage"""

    past_date = datetime.today() - timedelta(days=days)
    ticker_data = yf.Ticker(ticker)
    history = ticker_data.history(start=past_date)
    old_price = history.iloc[0]["Close"]
    current_price = history.iloc[-1]["Close"]
    return {"percent_change": ((current_price - old_price) / old_price) * 100}

In [11]:
get_current_stock_price('MSFT')

{'price': 344.6099853515625, 'currency': 'USD'}

In [12]:
get_stock_performance('MSFT', 30)

{'percent_change': 1.5770461202264492}

In [13]:
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class CurrentStockPriceInput(BaseModel):
    """Inputs for get_current_stock_price"""
    ticker: str = Field(description="Ticker symbol of the stock")

class CurrentStockPriceTool(BaseTool):
    name = "get_current_stock_price"
    description = """
        Useful when you want to get current stock price.
        You should enter the stock ticker symbol recognized by the yahoo finance
        """
    args_schema: Type[BaseModel] = CurrentStockPriceInput

    def _run(self, ticker: str):
        price_response = get_current_stock_price(ticker)
        return price_response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_current_stock_price does not support async")


class StockPercentChangeInput(BaseModel):
    """Inputs for get_stock_performance"""
    ticker: str = Field(description="Ticker symbol of the stock")
    days: int = Field(description='Timedelta days to get past date from current date')

class StockPerformanceTool(BaseTool):
    name = "get_stock_performance"
    description = """
        Useful when you want to check performance of the stock.
        You should enter the stock ticker symbol recognized by the yahoo finance.
        You should enter days as number of days from today from which performance needs to be check.
        output will be the change in the stock price represented as a percentage.
        """
    args_schema: Type[BaseModel] = StockPercentChangeInput

    def _run(self, ticker: str, days: int):
        response = get_stock_performance(ticker, days)
        return response

    def _arun(self, ticker: str):
        raise NotImplementedError("get_stock_performance does not support async")

In [14]:
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0613",
    temperature=0
)

tools = [
    CurrentStockPriceTool(),
    StockPerformanceTool()
]

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

In [15]:
agent.run("What is the current price of Microsoft stock? How it has performed over past 6 months?")



> Entering new  chain...

Invoking: `get_current_stock_price` with `{'ticker': 'MSFT'}`


{'price': 344.79998779296875, 'currency': 'USD'}
Invoking: `get_stock_performance` with `{'ticker': 'MSFT', 'days': 180}`


{'percent_change': 43.23662213490922}The current price of Microsoft stock is $344.80 USD. 

Over the past 6 months, Microsoft stock has performed well with a 43.24% increase in its price.

> Finished chain.


'The current price of Microsoft stock is $344.80 USD. \n\nOver the past 6 months, Microsoft stock has performed well with a 43.24% increase in its price.'

In [17]:
agent.run("Give me recent stock prices of Google and Meta?")



> Entering new  chain...

Invoking: `get_current_stock_price` with `{'ticker': 'GOOGL'}`


{'price': 125.18000030517578, 'currency': 'USD'}
Invoking: `get_current_stock_price` with `{'ticker': 'META'}`


{'price': 308.2699890136719, 'currency': 'USD'}The recent stock price of Google (GOOGL) is $125.18 USD, and the recent stock price of Meta (META) is $308.27 USD.

> Finished chain.


'The recent stock price of Google (GOOGL) is $125.18 USD, and the recent stock price of Meta (META) is $308.27 USD.'

In [18]:
agent.run('In the past 3 months, which stock between Microsoft and Google has performed the best?')



> Entering new  chain...

Invoking: `get_stock_performance` with `{'ticker': 'MSFT', 'days': 90}`


{'percent_change': 18.90896047721661}
Invoking: `get_stock_performance` with `{'ticker': 'GOOGL', 'days': 90}`


{'percent_change': 17.523827008502703}In the past 3 months, Microsoft (MSFT) has performed better than Google (GOOGL). Microsoft's stock price has increased by 18.91%, while Google's stock price has increased by 17.52%.

> Finished chain.


"In the past 3 months, Microsoft (MSFT) has performed better than Google (GOOGL). Microsoft's stock price has increased by 18.91%, while Google's stock price has increased by 17.52%."